In [47]:
import lightgbm
import pandas as pd

In [87]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


In [51]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

In [52]:
print(init_record.shape)
init_record['response_variable']

(123311, 339)


_key_client_id                        _key_occurreddate_month
0000D172-EA88-432F-8235-9FAA00D29072  2019-04-01                 0
                                      2019-05-01                 0
                                      2019-06-01                 0
                                      2019-07-01                 0
                                      2019-08-01                 0
                                                                ..
FFFF7C4D-F6DD-46D0-A9C0-AE3C2DC51132  2019-09-01                 0
                                      2019-10-01                 0
                                      2019-11-01                 0
                                      2019-12-01                 0
                                      2020-01-01                 0
Name: response_variable, Length: 123311, dtype: int64

In [53]:
def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    last_month_y = list(data.groupby(level=0).apply(lambda group: group.iloc[-1])['response_variable'].astype('int32'))
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable'].astype('int32')
    x = data.drop('response_variable',axis='columns')
    return x, y, last_month_y

feature, y, last_month_y = construct_feature_label(init_record)

## Use tsfresh to extract features

In [54]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.distribution import MultiprocessingDistributor
import re
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
from sklearn import metrics
import matplotlib.pyplot as plt

In [55]:
feature.columns = [re.sub(r"__*", "_", name) for name in feature.columns]

In [58]:
feature = feature.reset_index()

In [59]:
Distributor = MultiprocessingDistributor(n_workers=4,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")
settings = MinimalFCParameters()

In [60]:
extracted = extract_features(feature, column_id="_key_client_id", column_sort="_key_occurreddate_month",
                                   default_fc_parameters=settings, distributor=Distributor)

Feature Extraction: 100%|██████████| 20/20 [1:08:40<00:00, 206.03s/it]


In [61]:
from sklearn.ensemble import IsolationForest

In [62]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(extracted)

IsolationForest(contamination=0.1, n_estimators=50)

In [63]:
y_hat = model.predict(extracted)

In [64]:
y_hat = np.where(y_hat == -1, 1, 0)  

In [66]:
f1_score(y_hat, last_month_y)

0.22261904761904763

In [ ]:
confusion_matrix(y, last_month_y)

# Select certain columns

In [90]:
selected = init_record[init_record.filter(regex=r'allrows', axis=1).columns]
selected
selected = pd.concat([selected, init_record['response_variable'], init_record['_key_client_id'], init_record['_key_occurreddate_month']], axis=1)
selected
selected['_key_occurreddate_month']=selected['_key_occurreddate_month'].astype("datetime64[ns]")
selected = selected.set_index(['_key_client_id', '_key_occurreddate_month'])


In [91]:
def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    last_month_y = list(data.groupby(level=0).apply(lambda group: group.iloc[-1])['response_variable'].astype('int32'))
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable'].astype('int32')
    x = data.drop('response_variable',axis='columns')
    return x, y, last_month_y

selected_feature, selected_y, selected_last_month_y = construct_feature_label(selected)

In [92]:
selected_feature.shape

(109906, 60)

In [93]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.distribution import MultiprocessingDistributor
import re
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
from sklearn import metrics
import matplotlib.pyplot as plt

In [94]:
selected_feature.columns = [re.sub(r"__*", "_", name) for name in selected_feature.columns]

In [95]:
selected_feature = selected_feature.reset_index()

In [96]:
Distributor = MultiprocessingDistributor(n_workers=4,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")
settings = EfficientFCParameters()

In [ ]:
extracted = extract_features(selected_feature, column_id="_key_client_id", column_sort="_key_occurreddate_month",
                                   default_fc_parameters=settings, distributor=Distributor)

Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(extracted)

In [ ]:
y_hat = model.predict(extracted)
y_hat = np.where(y_hat == -1, 1, 0)  

In [ ]:
f1_score(y_hat, last_month_y)